<a href="https://colab.research.google.com/github/peremartra/LLMOptCost/blob/main/00/03_KnowledgeDistillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, AdamW
from datasets import load_dataset
from torch.nn import functional as F
from torch.utils.data import DataLoader

In [3]:
# Load teacher and student models and their tokenizers
teacher_model_name = "bigscience/bloomz-560m"
student_model_name = "oopere/bloomz-560m-pruned"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(teacher_model_name)

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [5]:
teacher_model = AutoModelForCausalLM.from_pretrained(teacher_model_name)
student_model = AutoModelForCausalLM.from_pretrained(student_model_name)

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/807 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [6]:
# Load the dataset
dataset = load_dataset('oopere/knowledge_transfer_1500_base')

README.md:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

kt2_1300.csv:   0%|          | 0.00/64.7k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1267 [00:00<?, ? examples/s]

In [7]:
# Prepare the dataloader
def tokenize_function(examples):
    return tokenizer(examples['Prompt'], padding="max_length", truncation=True, max_length=128)

In [8]:
tokenized_datasets = dataset['train'].map(tokenize_function, batched=True, remove_columns=["Prompt", "Topic"])
# Convert to PyTorch format
tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask'])

Map:   0%|          | 0/1267 [00:00<?, ? examples/s]

In [9]:
# Create DataLoader
dataloader = DataLoader(tokenized_datasets, batch_size=8)

In [10]:
# Move models to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
teacher_model.to(device)
student_model.to(device)


BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 1024)
    (word_embeddings_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-23): 24 x BloomBlock(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (

In [11]:
# Set the teacher model to evaluation mode
teacher_model.eval()

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 1024)
    (word_embeddings_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-23): 24 x BloomBlock(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (

In [15]:
# Define optimizer for student model
optimizer = AdamW(student_model.parameters(), lr=2e-5)

In [21]:
# Training loop
num_epochs = 10
temperature = 1.5  # Temperature for soft targets
alpha = 1  # All to soft loss.


In [22]:
for epoch in range(num_epochs):
    student_model.train()
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        # Get teacher model logits
        with torch.no_grad():
            teacher_outputs = teacher_model(input_ids, attention_mask=attention_mask)
            teacher_logits = teacher_outputs.logits / temperature

        # Get student model logits
        student_outputs = student_model(input_ids, attention_mask=attention_mask)
        student_logits = student_outputs.logits

        # Compute soft target loss using KL-divergence
        teacher_probs = F.softmax(teacher_logits, dim=-1)
        student_log_probs = F.log_softmax(student_logits / temperature, dim=-1)
        loss = F.kl_div(student_log_probs, teacher_probs, reduction="batchmean") * (temperature ** 2)

        # Backpropagate and optimize
        loss.backward()

        #graddient clipping to aviod inestability in final epochs of training.
        #torch.nn.utils.clip_grad_norm_(student_model.parameters(), max_norm=1.0)

        optimizer.step()
        optimizer.zero_grad()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")


Epoch 1/10, Loss: 14.121241569519043
Epoch 2/10, Loss: 14.11896800994873
Epoch 3/10, Loss: 14.117864608764648
Epoch 4/10, Loss: 14.115164756774902
Epoch 5/10, Loss: 14.114659309387207
Epoch 6/10, Loss: 14.11430549621582
Epoch 7/10, Loss: 14.113114356994629
Epoch 8/10, Loss: 14.109735488891602
Epoch 9/10, Loss: 14.110427856445312
Epoch 10/10, Loss: 14.110933303833008


In [25]:
student_model_name = "bloomz-560m-pruned-kd"

In [26]:
# Save the fine-tuned student model
student_model.save_pretrained(student_model_name)
tokenizer.save_pretrained(student_model_name)

('bloomz-560m-pruned-kd/tokenizer_config.json',
 'bloomz-560m-pruned-kd/special_tokens_map.json',
 'bloomz-560m-pruned-kd/tokenizer.json')

In [28]:
!pip install -q huggingface_hub

In [29]:
student_model.push_to_hub(student_model_name,
                  private=False,
                  use_temp_dir=False)
tokenizer.push_to_hub(student_model_name,
                      private=False,
                      use_temp_dir=False)

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/oopere/bloomz-560m-pruned-kd/commit/4e91d266831b86a49fd31ec160bb9b7aa502dfb3', commit_message='Upload tokenizer', commit_description='', oid='4e91d266831b86a49fd31ec160bb9b7aa502dfb3', pr_url=None, pr_revision=None, pr_num=None)